In [ ]:
# step 1-> identify oi cross over in trending OI

# first i need oi data of all strikes
# then i will select the trending oi 
# then I will do the sum and check the cross over happening or not 

# Always there are two things 
# 1. check on historical data 
#     for this i need data and i will loop over each date and check the condition 
#         if condition meet print the datetime
# 2. check in live market 
#     fetch the live data and do the same computation 


In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
from kite_trade import *
from enctoken import get_kite
kite = get_kite()
import warnings
warnings.filterwarnings(action = 'ignore')
from IPython.display import clear_output
import datetime

# oi data

In [ ]:
def fetch_market_data(expiry_march, start_dt, end_dt, time_frame="minute"):
    df_expiry_hist_data = pd.DataFrame()
    strike_symbol_dict = dict(
        zip(expiry_march.instrument_token, expiry_march["strike_type"])
    )
    inst_expiry = expiry_march["instrument_token"].unique().tolist()
    for i in inst_expiry:
        df = pd.DataFrame(
            kite.historical_data(
                i,
                from_date=start_dt,
                to_date=end_dt,
                interval=time_frame,
                continuous=False,
                oi=True,
            )
        )
        df["strike_type"] = strike_symbol_dict[i]
        df_expiry_hist_data = df_expiry_hist_data.append(df)
    df_expiry_hist_data["date"] = pd.to_datetime(
        df_expiry_hist_data["date"]
    ).dt.tz_localize(None)
    df_expiry_hist_data["date_only"] = pd.to_datetime(
        df_expiry_hist_data["date"].dt.date
    )
    return df_expiry_hist_data


def reorder_column(df):
    strike_prices = sorted(
        set(int(col.split("_")[1]) for col in df.columns if "oi" or "chg" in col)
    )
    reordered_columns = []
    for price in strike_prices:
        reordered_columns.extend(
            [f"oi_{price}_CE", f"chg_{price}_CE",f"chg_{price}_PE", f"oi_{price}_PE" ]
        )
    return reordered_columns


def prev_day_oi(oi_strike, itm_strike, atm_strike, otm_strike):
    for i in range(1, 6):
        day = datetime.datetime.now() - datetime.timedelta(days=i)
        start_dt_prev = day.strftime("%Y-%m-%d")
        end_dt_prev = day.strftime("%Y-%m-%d")
        time_frame = "minute"
        print(start_dt_prev, end_dt_prev)
        df_prev = fetch_market_data(oi_strike, start_dt_prev, end_dt_prev, time_frame)
        print(df_prev.shape)
        if df_prev.empty:
            continue
        else:
            df_prev["strike"] = df_prev["strike_type"].str[:-2].astype(int)
            df_prev["ce_pe"] = df_prev["strike_type"].str[-2:]
            break
    df_prev = (
        df_prev[df_prev["strike"].isin(itm_strike + [atm_strike] + otm_strike)]
        .groupby("strike_type")
        .tail(1)
    )
    return df_prev


In [ ]:
expiry_sp = 'BANKNIFTY2465'
start_dt = datetime.datetime.now().strftime("%Y-%m-%d")
end_dt = datetime.datetime.now().strftime("%Y-%m-%d")
time_frame = "minute"

inst = pd.DataFrame(kite.instruments("NFO"))
expiry_march  = inst[(inst.name == 'BANKNIFTY') & (inst.tradingsymbol.str.contains(expiry_sp))]
expiry_march['strike_type'] = expiry_march['strike'].astype(int).astype(str) + expiry_march['instrument_type']

In [125]:

df_prev = pd.DataFrame()
ltp_dict = kite.ltp([256265, 260105])
nifty_price = ltp_dict["256265"]["last_price"]
banknifty_price = ltp_dict["260105"]["last_price"]
atm_strike = int(banknifty_price // 100 * 100)
otm_strike = [atm_strike + i * 100 for i in range(1, 3)]
itm_strike = [atm_strike - i * 100 for i in range(1, 3)]
print(atm_strike, otm_strike, itm_strike)

oi_strike = expiry_march.query(
    "strike in @otm_strike or strike in @itm_strike or strike == @atm_strike"
)

if df_prev.empty:
    df_prev = prev_day_oi(oi_strike, itm_strike, atm_strike, otm_strike)

df_oi = fetch_market_data(oi_strike, start_dt, end_dt, time_frame)
df_oi["strike"] = df_oi["strike_type"].str[:-2].astype(int)
df_oi["ce_pe"] = df_oi["strike_type"].str[-2:]

df_oi_merge = pd.merge(
    df_oi, df_prev[["strike_type", "oi"]], on="strike_type", suffixes=("_live", "_prev")
).assign(chg=lambda x: x["oi_live"] - x["oi_prev"])
df_oi_merge = df_oi_merge.drop(["oi_prev"], axis=1).rename(columns={"oi_live": "oi"})
df_oi_merge_pivot = (
    df_oi_merge[df_oi_merge["strike"].isin(itm_strike + [atm_strike] + otm_strike)]
    .pivot_table(
        index="date", columns=["strike", "ce_pe"], values=["oi", "chg"], aggfunc="sum"
    )
    .sort_values("date", ascending=False)
)
df_oi_merge_pivot.columns = [
    "_".join([str(i) for i in col]) for col in df_oi_merge_pivot.columns
]
df_oi_merge_pivot[reorder_column(df_oi_merge_pivot)].head()


48900 [49000, 49100] [48800, 48700]
2024-04-09 2024-04-09
(3750, 9)


,oi_48700_CE,chg_48700_CE,chg_48700_PE,oi_48700_PE,oi_48800_CE,chg_48800_CE,chg_48800_PE,oi_48800_PE,oi_48900_CE,chg_48900_CE,chg_48900_PE,oi_48900_PE,oi_49000_CE,chg_49000_CE,chg_49000_PE,oi_49000_PE,oi_49100_CE,chg_49100_CE,chg_49100_PE,oi_49100_PE
date,,,,,,,,,,,,,,,,,,,,
2024-04-10 15:29:00,1004595,-662700,518850,2248275,1579215,-410970,2828250,3856410,3896610,1989720,6227400,6787830,9183495,6023670,7176885,7824705,4026435,2669790,1288545,1482570
2024-04-10 15:28:00,1004595,-662700,518850,2248275,1579215,-410970,2828250,3856410,3896610,1989720,6227400,6787830,9183495,6023670,7176885,7824705,4026435,2669790,1288545,1482570
2024-04-10 15:27:00,1047720,-619575,755325,2484750,1778220,-211965,3312675,4340835,4129140,2222250,7118130,7678560,9856290,6696465,8521425,9169245,4476360,3119715,1441470,1635495
2024-04-10 15:26:00,1047720,-619575,755325,2484750,1778220,-211965,3312675,4340835,4129140,2222250,7118130,7678560,9856290,6696465,8521425,9169245,4476360,3119715,1441470,1635495
2024-04-10 15:25:00,1047720,-619575,755325,2484750,1778220,-211965,3312675,4340835,4129140,2222250,7118130,7678560,9856290,6696465,8521425,9169245,4476360,3119715,1441470,1635495


In [127]:
df_oi.query("strike_type == '48800CE'").tail(5)


,date,open,high,low,close,volume,oi,strike_type,date_only,strike,ce_pe
370,2024-04-10 15:25:00,185.9,186.65,184.8,185.80,175560,1778220,48800CE,2024-04-10,48800,CE
371,2024-04-10 15:26:00,185.8,187.65,185.3,185.65,151590,1778220,48800CE,2024-04-10,48800,CE
372,2024-04-10 15:27:00,185.9,186.05,184.2,185.35,91170,1778220,48800CE,2024-04-10,48800,CE
373,2024-04-10 15:28:00,185.4,186.45,185.1,186.10,80580,1579215,48800CE,2024-04-10,48800,CE
374,2024-04-10 15:29:00,186.1,186.70,185.6,186.40,85245,1579215,48800CE,2024-04-10,48800,CE


# live market oi cross over

In [2]:
expiry_sp = 'BANKNIFTY24410'
inst = pd.DataFrame(kite.instruments("NFO"))
expiry_march  = inst[(inst.name == 'BANKNIFTY') & (inst.tradingsymbol.str.contains(expiry_sp))]
# expiry_march = expiry_march.query("(strike >= 38500 )&( strike <= 40500)")
expiry_march['strike_type'] = expiry_march['strike'].astype(int).astype(str) + expiry_march['instrument_type']


In [3]:
expiry_march.head()

,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange,strike_type
2255,11799298,46091,BANKNIFTY2441039000CE,BANKNIFTY,0.0,2024-04-10,39000.0,0.05,15,CE,NFO-OPT,NFO,39000CE
2256,11799554,46092,BANKNIFTY2441039000PE,BANKNIFTY,0.0,2024-04-10,39000.0,0.05,15,PE,NFO-OPT,NFO,39000PE
2257,11800578,46096,BANKNIFTY2441039500CE,BANKNIFTY,0.0,2024-04-10,39500.0,0.05,15,CE,NFO-OPT,NFO,39500CE
2258,11800834,46097,BANKNIFTY2441039500PE,BANKNIFTY,0.0,2024-04-10,39500.0,0.05,15,PE,NFO-OPT,NFO,39500PE
2259,11801090,46098,BANKNIFTY2441040000CE,BANKNIFTY,0.0,2024-04-10,40000.0,0.05,15,CE,NFO-OPT,NFO,40000CE


In [ ]:
# for today's oi cross over check 
# 1. i need the todays data and continously fetch the 1 day before data and take the last day ending data
# 2. then i will check the trending oi cross over 
# 3. then i will check strike cross over 
#     for it i need current spot price
#     and i will check in 1 above and below strike price for cross over 
#     i will plot the 3 strike and keep the oi table if needed


In [37]:
import datetime
import time

def fetch_market_data(expiry_march,start_dt, end_dt, time_frame = 'minute'):
    df_expiry_hist_data = pd.DataFrame()
    strike_symbol_dict = dict(zip(expiry_march.instrument_token,expiry_march['strike_type'] ))
    inst_expiry = expiry_march['instrument_token'].unique().tolist() 
    for i in inst_expiry:
        df = pd.DataFrame(kite.historical_data(i, from_date = start_dt, to_date = end_dt, interval = time_frame, continuous=False, oi=True))
        df['strike_type'] = strike_symbol_dict[i]
        df_expiry_hist_data = df_expiry_hist_data.append(df)
    df_expiry_hist_data['date']  = pd.to_datetime(df_expiry_hist_data['date']).dt.tz_localize(None)
    df_expiry_hist_data['date_only'] = pd.to_datetime(df_expiry_hist_data['date'].dt.date)
    return df_expiry_hist_data



def get_previous_trading_day_data():
    # Check the last five days
    pass


# previous_day_data = get_previous_trading_day_data()
# print(f"Previous trading day's data at 15:27: {previous_day_data}")

# while True:
#     now = datetime.datetime.now()
#     today_data = fetch_market_data(now.strftime('%Y-%m-%d'), now.strftime('%H:%M'))
#     print(f"Today's data at {now.strftime('%H:%M')}: {today_data}")
    
#     # Wait for 60 seconds before the next fetch
#     time.sleep(60)



In [ ]:
import datetime
trending_strike = [46100, 46200, 46300, 46400, 46500, 46600, 46700, 46800, 46900, 47000, 47100, 47200, 47300, 47400, 47500]

# previous day data fetch 
for i in range(1, 6):
        day = datetime.datetime.now() - datetime.timedelta(days=i)
        start_dt = day.strftime('%Y-%m-%d')
        end_dt = day.strftime('%Y-%m-%d')
        time_frame = '3minute'
        df_prev = fetch_market_data(expiry_march,start_dt, end_dt, time_frame)
        if df_prev.empty:
            continue
        else:
               df_prev
        # break
               
    df_bn_oi_data['strike_value'] = df_bn_oi_data['strike_type'].str[:-2]
    df_bn_oi_data['strike_value'] = df_bn_oi_data['strike_value'].astype(int)
    df_bn_oi_data['strike_ce_pe'] = df_bn_oi_data['strike_type'].str[-2:]
    df_bn_oi_data.query('strike_value in @trending_strike', inplace = True)
    df_bn_oi_data_prev_day = df_bn_oi_data_prev_day.groupby(['date', 'strike_ce_pe'])['oi'].sum().reset_index()
    df_pivot_prev = df_bn_oi_data_prev_day.pivot_table(index='date', columns='strike_ce_pe', values='oi')

In [ ]:
# current day data fetch 
now = datetime.datetime.now()
start_dt = now.strftime('%Y-%m-%d')
end_dt = now.strftime('%Y-%m-%d')
time_frame = '3minute'
df_current_day = fetch_market_data(expiry_march,start_dt, end_dt, time_frame)
df_current_day['strike_value'] = df_current_day['strike_type'].str[:-2]
df_current_day['strike_value'] = df_current_day['strike_value'].astype(int)
df_current_day['strike_ce_pe'] = df_current_day['strike_type'].str[-2:]
df_current_day.query('strike_value in @trending_strike', inplace = True)
df_bn_oi_data_current_day = df_current_day.groupby(['date', 'strike_ce_pe'])['oi'].sum().reset_index()
df_pivot_current = df_bn_oi_data_current_day.pivot_table(index='date', columns='strike_ce_pe', values='oi')

In [ ]:
## cross over 
df_pivot = df_pivot_curr.reset_index()
df_pivot.drop(['CE', 'PE'], axis = 1, inplace = True)

# Initialize an empty list to store crossover points
crossovers = []

# Loop through the DataFrame starting from the second row
for i in range(1, len(df_pivot)):
    # Current and previous values for 'chg_ce' and 'chg_pe'
    current_chg_ce = df_pivot.loc[i, 'chg_ce']
    previous_chg_ce = df_pivot.loc[i - 1, 'chg_ce']
    current_chg_pe = df_pivot.loc[i, 'chg_pe']
    previous_chg_pe = df_pivot.loc[i - 1, 'chg_pe']

    # Check for crossover: chg_ce crosses above chg_pe or chg_ce crosses below chg_pe
    if (current_chg_ce > current_chg_pe and previous_chg_ce < previous_chg_pe) or \
       (current_chg_ce < current_chg_pe and previous_chg_ce > previous_chg_pe):
        # Crossover detected; add it to the list
        crossover_date = df_pivot.loc[i, 'date']
        crossovers.append(crossover_date)

# Print all crossover points
for crossover in crossovers:
    print(f"Crossover occurred on: {crossover}")

# If you want to return the crossovers as well, just use `return crossovers` if this is in a function.

In [44]:
from plyer import notification

# Your condition
condition = True  # This is just an example condition. Replace it with your actual condition.

if condition:
    notification.notify(
        title='Akash first notification',
        message='Welcome to trading world',
        app_name='Akash Trading App',
    )

## oi cross over 

In [2]:
df_bn_oi_data = pd.read_parquet("../data/expiry_27march2024_3min.parquet")
df_bn_oi_data = df_bn_oi_data.query("strike_type != '0FUT'")
df_bn_oi_data['date']  = pd.to_datetime(df_bn_oi_data['date']).dt.tz_localize(None)
df_bn_oi_data['date_only'] = pd.to_datetime(df_bn_oi_data['date'].dt.date)
# trending_strike = list(range(46100, 47501, 100))
trending_strike = [46100, 46200, 46300, 46400, 46500, 46600, 46700, 46800, 46900, 47000, 47100, 47200, 47300, 47400, 47500]
# trending_strike = [46700]
df_bn_oi_data['strike_value'] = df_bn_oi_data['strike_type'].str[:-2]
df_bn_oi_data['strike_value'] = df_bn_oi_data['strike_value'].astype(int)
df_bn_oi_data['strike_ce_pe'] = df_bn_oi_data['strike_type'].str[-2:]
df_bn_oi_data.query('strike_value in @trending_strike', inplace = True)

In [3]:
df_bn_oi_data_prev_day = df_bn_oi_data.query("date_only == '2024-03-21'")
df_bn_oi_data_prev_day = df_bn_oi_data_prev_day.groupby(['date', 'strike_ce_pe'])['oi'].sum().reset_index()
df_pivot_prev = df_bn_oi_data_prev_day.pivot_table(index='date', columns='strike_ce_pe', values='oi')

In [4]:
df_pivot_prev.tail()

strike_ce_pe,CE,PE
date,,
2024-03-21 15:15:00,13427790.0,10294725.0
2024-03-21 15:18:00,13233225.0,10125030.0
2024-03-21 15:21:00,12830535.0,9773445.0
2024-03-21 15:24:00,12498615.0,9357075.0
2024-03-21 15:27:00,12304725.0,8987835.0


In [5]:
prev_day_ce_oi = 12304725.0 - (5520975 - 4935675)
prev_day_pe_oi = 8987835.0 - (4918185 - 4573980)

In [6]:
df_bn_oi_data_curr_day = df_bn_oi_data.query("date_only == '2024-03-22'")
df_bn_oi_data_curr_day= df_bn_oi_data_curr_day.groupby(['date', 'strike_ce_pe'])['oi'].sum().reset_index()
df_pivot_curr = df_bn_oi_data_curr_day.pivot_table(index='date', columns='strike_ce_pe', values='oi')

In [7]:
df_pivot_curr['chg_ce'] = df_pivot_curr['CE'] - prev_day_ce_oi
df_pivot_curr['chg_pe'] = df_pivot_curr['PE'] -prev_day_pe_oi

In [8]:
df_pivot_curr.head()

strike_ce_pe,CE,PE,chg_ce,chg_pe
date,,,,
2024-03-22 09:15:00,12028170.0,8963310.0,308745.0,319680.0
2024-03-22 09:18:00,13238385.0,10088415.0,1518960.0,1444785.0
2024-03-22 09:21:00,13866630.0,10891080.0,2147205.0,2247450.0
2024-03-22 09:24:00,14345490.0,11390700.0,2626065.0,2747070.0
2024-03-22 09:27:00,14617845.0,11503155.0,2898420.0,2859525.0


In [9]:
import pandas as pd

df_pivot = df_pivot_curr.reset_index()
df_pivot.drop(['CE', 'PE'], axis = 1, inplace = True)

# Initialize an empty list to store crossover points
crossovers = []

# Loop through the DataFrame starting from the second row
for i in range(1, len(df_pivot)):
    # Current and previous values for 'chg_ce' and 'chg_pe'
    current_chg_ce = df_pivot.loc[i, 'chg_ce']
    previous_chg_ce = df_pivot.loc[i - 1, 'chg_ce']
    current_chg_pe = df_pivot.loc[i, 'chg_pe']
    previous_chg_pe = df_pivot.loc[i - 1, 'chg_pe']

    # Check for crossover: chg_ce crosses above chg_pe or chg_ce crosses below chg_pe
    if (current_chg_ce > current_chg_pe and previous_chg_ce < previous_chg_pe) or \
       (current_chg_ce < current_chg_pe and previous_chg_ce > previous_chg_pe):
        # Crossover detected; add it to the list
        crossover_date = df_pivot.loc[i, 'date']
        crossovers.append(crossover_date)

# Print all crossover points
for crossover in crossovers:
    print(f"Crossover occurred on: {crossover}")

# If you want to return the crossovers as well, just use `return crossovers` if this is in a function.


Crossover occurred on: 2024-03-22 09:18:00
Crossover occurred on: 2024-03-22 09:21:00
Crossover occurred on: 2024-03-22 09:27:00
Crossover occurred on: 2024-03-22 09:36:00


In [10]:
import plotly.express as px
import pandas as pd

# Assuming df_bn_oi_data is your DataFrame with the necessary data
# df_pivot = df_bn_oi_data.groupby(['date', 'strike_ce_pe'])['oi'].sum().reset_index().pivot(index='date', columns='strike_ce_pe', values='oi')

# Reset index to make 'date' a column again, which is necessary for Plotly


# Plotting with Plotly
fig = px.line(df_pivot, x='date', y=df_pivot.columns,
              title='OI Pulse',
              labels={'value': 'OI', 'variable': 'Option Type'})

# Adding more interactive features like hover information
fig.update_traces(mode='lines+markers')
fig.update_layout(hovermode='x unified')

fig.show()


In [73]:
df_pivot

strike_ce_pe,date,CE,PE
0,2024-03-22 09:15:00,12028170.0,8963310.0
1,2024-03-22 09:18:00,13238385.0,10088415.0
2,2024-03-22 09:21:00,13866630.0,10891080.0
3,2024-03-22 09:24:00,14345490.0,11390700.0
4,2024-03-22 09:27:00,14617845.0,11503155.0
...,...,...,...
120,2024-03-22 15:15:00,12484230.0,14412510.0
121,2024-03-22 15:18:00,12233190.0,14141160.0
122,2024-03-22 15:21:00,11942310.0,13658610.0
123,2024-03-22 15:24:00,11577585.0,13068345.0


In [58]:
import pandas as pd
import plotly.graph_objs as go

# Assuming df_pivot is your DataFrame with 'date', 'CE', and 'PE' columns

# Convert 'date' column to datetime
df_pivot['date'] = pd.to_datetime(df_pivot['date'])

# Sort the DataFrame by 'date' just in case it's not sorted
df_pivot = df_pivot.sort_values('date')

# Calculate the change in 'CE' and 'PE'
df_pivot['change_in_CE'] = df_pivot['CE'].diff()
df_pivot['change_in_PE'] = df_pivot['PE'].diff()

# Create a Plotly graph
fig = go.Figure()

# Add traces for 'change_in_CE' and 'change_in_PE'
fig.add_trace(go.Scatter(x=df_pivot['date'], y=df_pivot['change_in_CE'],
                         mode='lines+markers', name='Change in Call OI'))
fig.add_trace(go.Scatter(x=df_pivot['date'], y=df_pivot['change_in_PE'],
                         mode='lines+markers', name='Change in Put OI'))

# Update layout for a better look
fig.update_layout(title='Change in OI for CE and PE',
                  xaxis_title='Date',
                  yaxis_title='Change in OI',
                  hovermode='x unified')

# Show plot
fig.show()


In [56]:
df_pivot

strike_ce_pe,date,CE,PE
0,2024-03-21 09:15:00,7026255.0,4429050.0
1,2024-03-21 09:18:00,7703340.0,5392590.0
2,2024-03-21 09:21:00,8208375.0,6696285.0
3,2024-03-21 09:24:00,8786385.0,7280700.0
4,2024-03-21 09:27:00,9250770.0,7973070.0
...,...,...,...
120,2024-03-21 15:15:00,13427790.0,10294725.0
121,2024-03-21 15:18:00,13233225.0,10125030.0
122,2024-03-21 15:21:00,12830535.0,9773445.0
123,2024-03-21 15:24:00,12498615.0,9357075.0


In [49]:
df_bn_oi_data.query("strike_value == 46600 & strike_ce_pe == 'CE'").tail(10)

,date,open,high,low,close,volume,oi,strike_type,date_only,strike_value,strike_ce_pe
115,2024-03-21 15:00:00,477.85,481.85,459.75,467.70,133020.0,592890.0,46600CE,2024-03-21,46600,CE
116,2024-03-21 15:03:00,466.25,472.90,461.15,468.75,90975.0,602280.0,46600CE,2024-03-21,46600,CE
117,2024-03-21 15:06:00,468.25,469.95,445.95,459.90,158145.0,601545.0,46600CE,2024-03-21,46600,CE
118,2024-03-21 15:09:00,459.90,471.95,459.40,469.75,128505.0,593700.0,46600CE,2024-03-21,46600,CE
119,2024-03-21 15:12:00,469.85,471.00,462.00,466.90,78765.0,582495.0,46600CE,2024-03-21,46600,CE
120,2024-03-21 15:15:00,466.50,470.65,460.90,467.00,101475.0,558465.0,46600CE,2024-03-21,46600,CE
121,2024-03-21 15:18:00,467.40,469.45,461.40,462.60,80955.0,535725.0,46600CE,2024-03-21,46600,CE
122,2024-03-21 15:21:00,462.95,467.00,460.60,464.15,81375.0,504615.0,46600CE,2024-03-21,46600,CE
123,2024-03-21 15:24:00,464.15,465.50,458.40,463.60,92700.0,471855.0,46600CE,2024-03-21,46600,CE
124,2024-03-21 15:27:00,463.60,471.05,460.00,470.95,86055.0,455295.0,46600CE,2024-03-21,46600,CE


In [2]:
expiry_sp = 'BANKNIFTY24MAR'
inst = pd.DataFrame(kite.instruments("NFO"))
expiry_march  = inst[(inst.name == 'BANKNIFTY') & (inst.tradingsymbol.str.contains(expiry_sp))]
# expiry_march = expiry_march.query("(strike >= 38500 )&( strike <= 40500)")
expiry_march['strike_type'] = expiry_march['strike'].astype(int).astype(str) + expiry_march['instrument_type']

In [24]:
inst.query("name =='NIFTY' ")

,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange
0,13368834,52222,NIFTY24APRFUT,NIFTY,0.0,2024-04-25,0.0,0.05,50,FUT,NFO-FUT,NFO
1,9372674,36612,NIFTY24MARFUT,NIFTY,0.0,2024-03-28,0.0,0.05,50,FUT,NFO-FUT,NFO
2,12014082,46930,NIFTY24MAYFUT,NIFTY,0.0,2024-05-30,0.0,0.05,50,FUT,NFO-FUT,NFO
558,15440642,60315,NIFTY24MAR14000CE,NIFTY,0.0,2024-03-28,14000.0,0.05,50,CE,NFO-OPT,NFO
559,15440898,60316,NIFTY24MAR14000PE,NIFTY,0.0,2024-03-28,14000.0,0.05,50,PE,NFO-OPT,NFO
...,...,...,...,...,...,...,...,...,...,...,...,...
2076,17352962,67785,NIFTY28DEC25000PE,NIFTY,0.0,2028-12-28,25000.0,0.05,50,PE,NFO-OPT,NFO
2077,17353218,67786,NIFTY28DEC26000CE,NIFTY,0.0,2028-12-28,26000.0,0.05,50,CE,NFO-OPT,NFO
2078,17353474,67787,NIFTY28DEC26000PE,NIFTY,0.0,2028-12-28,26000.0,0.05,50,PE,NFO-OPT,NFO
2079,17353730,67788,NIFTY28DEC27000CE,NIFTY,0.0,2028-12-28,27000.0,0.05,50,CE,NFO-OPT,NFO


In [9]:
i = 9372674 #9372418 #
start_dt = '2024-03-01'
end_dt = '2024-03-27'
time_frame = '3minute'
df = pd.DataFrame(kite.historical_data(i, from_date = start_dt, to_date = end_dt, interval = time_frame, continuous=False, oi=True))

In [11]:
df.to_parquet('data/nifty_march_futures.parquet')

In [85]:
df['date']  = pd.to_datetime(df['date']).dt.tz_localize(None)
df['date_only'] = pd.to_datetime(df['date'].dt.date)

## gap theory

In [ ]:
import pandas as pd

# Assuming 'df' is your DataFrame with columns: date, open, high, low, close

# Convert the 'date' column to datetime format and filter the entries between 9:30 and 15:00
# df['date'] = pd.to_datetime(df['date'])
# filter for date 20/03/2024



# Initialize an empty list to store the gap information
gaps = []

# Iterate over the filtered DataFrame to find gaps
for j in df.date_only.unique():
    print(j)
    df_temp = df[df.date_only == j]
    filtered_df = df_temp[df_temp['date'].dt.time.between(pd.to_datetime('09:30').time(), pd.to_datetime('15:00').time())].reset_index(drop = True)
    for i in range(1,len(filtered_df)):
        previous_row = filtered_df.iloc[i - 1]
        current_row = filtered_df.iloc[i]

        # Check if the previous candle is green
        if previous_row['close'] > previous_row['open']:
            # Check for a gap up
            if current_row['low'] > previous_row['high'] + 2:
                gaps.append({
                    'type': 'gap up',
                    'from': previous_row['high'],
                    'to': current_row['open'],
                    'index': i  # Index of the current row in the filtered DataFrame
                })

        # Check if the previous candle is red
        elif previous_row['close'] < previous_row['open']:
            # Check for a gap down
            if current_row['high'] < previous_row['low'] - 2:
                gaps.append({
                    'type': 'gap down',
                    'from': previous_row['low'],
                    'to': current_row['open'],
                    'index': i  # Index of the current row in the filtered DataFrame
                })
        # break
    
# The 'gaps' list now contains the details of all identified gaps


## golden cross over

In [79]:
df.head()

,date,open,high,low,close,volume,oi
0,2024-03-01 09:15:00+05:30,22203.00,22250.00,22180.20,22243.95,357550,13084800
1,2024-03-01 09:18:00+05:30,22243.95,22244.95,22232.45,22239.75,140950,13069950
2,2024-03-01 09:21:00+05:30,22239.75,22248.60,22236.20,22242.30,120400,13070650
3,2024-03-01 09:24:00+05:30,22242.30,22254.90,22240.80,22248.20,129050,13080800
4,2024-03-01 09:27:00+05:30,22248.20,22253.45,22238.20,22245.00,100950,13027100


In [86]:
import pandas as pd
import numpy as np
import ta

def calculate_supertrend(df, period=10, multiplier=2):
    hl2 = (df['high'] + df['low']) / 2
    atr = ta.volatility.average_true_range(df['high'], df['low'], df['close'], window=period)
    upperband = hl2 + (multiplier * atr)
    lowerband = hl2 - (multiplier * atr)
    supertrend = np.zeros(len(df))

    for i in range(1, len(df)):
        if df['close'][i-1] <= supertrend[i-1]:
            supertrend[i] = min(upperband[i], supertrend[i-1])
        else:
            supertrend[i] = max(lowerband[i], supertrend[i-1])
        if df['close'][i] > supertrend[i]:
            supertrend[i] = upperband[i]
        elif df['close'][i] < supertrend[i]:
            supertrend[i] = lowerband[i]

    return supertrend

def calculate_daily_vwap(df):
    df['cum_vol_price'] = (df['volume'] * df['close']).cumsum()
    df['cum_volume'] = df['volume'].cumsum()
    df['VWAP'] = df['cum_vol_price'] / df['cum_volume']
    return df

def calculate_indicators(df):
    df['VWMA20'] = ta.trend.sma_indicator(df['close'] * df['volume'], window=20) / ta.trend.sma_indicator(df['volume'], window=20)
    df = df.groupby('date_only').apply(calculate_daily_vwap)
    df['SuperTrend'] = calculate_supertrend(df, period=10, multiplier=2)
    return df

def generate_signals(df):
    df = calculate_indicators(df)
    df['Signal'] = ((df['VWMA20'] > df['VWAP']) & (df['SuperTrend'] > df['VWAP'])) | ((df['VWMA20'] < df['VWAP']) & (df['SuperTrend'] < df['VWAP']))
    # signals = df[df['Signal']].copy()
    return df


signals = generate_signals(df)
# print(signals)


In [88]:
signals.tail(10)

,date,open,high,low,close,volume,oi,date_only,VWMA20,cum_vol_price,cum_volume,VWAP,SuperTrend,Signal
1650,2024-03-20 15:00:00,21915.00,21916.40,21896.20,21897.55,103100,11906750,2024-03-20,21907.435977,1.305767e+11,5960400,21907.376540,21868.013702,False
1651,2024-03-20 15:03:00,21897.00,21907.95,21889.70,21898.90,46350,11917600,2024-03-20,21906.646546,1.315917e+11,6006750,21907.311132,21936.932668,False
1652,2024-03-20 15:06:00,21901.00,21908.70,21900.00,21908.40,47350,11944150,2024-03-20,21905.804739,1.326291e+11,6054100,21907.319649,21868.093098,True
1653,2024-03-20 15:09:00,21906.35,21914.00,21906.35,21910.00,42700,11954300,2024-03-20,21905.954934,1.335647e+11,6096800,21907.338421,21944.336211,False
1654,2024-03-20 15:12:00,21910.00,21919.60,21908.75,21914.90,42500,11968850,2024-03-20,21905.966957,1.344960e+11,6139300,21907.390767,21881.259910,True
1655,2024-03-20 15:15:00,21915.65,21918.30,21910.50,21917.00,52450,11990050,2024-03-20,21905.655598,1.356456e+11,6191750,21907.472166,21945.583581,False
1656,2024-03-20 15:18:00,21916.55,21923.90,21910.85,21916.50,33850,12003500,2024-03-20,21905.334387,1.363875e+11,6225600,21907.521253,21886.699777,True
1657,2024-03-20 15:21:00,21916.50,21922.60,21915.85,21916.65,38650,12015100,2024-03-20,21905.951501,1.372345e+11,6264250,21907.577576,21948.182701,False
1658,2024-03-20 15:24:00,21917.00,21922.20,21915.05,21916.00,37600,12017350,2024-03-20,21906.856145,1.380586e+11,6301850,21907.627829,21891.133069,True
1659,2024-03-20 15:27:00,21915.00,21920.00,21914.90,21919.00,50900,12015350,2024-03-20,21907.626953,1.391743e+11,6352750,21907.718946,21943.212738,False


## PRICE AT SUPER TREND AND VWAP

In [99]:
def date_format(df):
    df['date']  = pd.to_datetime(df['date']).dt.tz_localize(None)
    df['date_only'] = pd.to_datetime(df['date'].dt.date)
    return df 

In [110]:
import talib
expiry_sp = 'BANKNIFTY24430'
inst = pd.DataFrame(kite.instruments("NFO"))
expiry_march  = inst[(inst.name == 'BANKNIFTY') & (inst.tradingsymbol.str.contains(expiry_sp))]
# expiry_march = expiry_march.query("(strike >= 38500 )&( strike <= 40500)")
expiry_march['strike_type'] = expiry_march['strike'].astype(int).astype(str) + expiry_march['instrument_type']

In [118]:
expiry_march[expiry_march['tradingsymbol'].str.contains("48200")]

,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange,strike_type
2059,12679938,49531,BANKNIFTY2443048200CE,BANKNIFTY,0.0,2024-04-30,48200.0,0.05,15,CE,NFO-OPT,NFO,48200CE
2060,12680194,49532,BANKNIFTY2443048200PE,BANKNIFTY,0.0,2024-04-30,48200.0,0.05,15,PE,NFO-OPT,NFO,48200PE


In [119]:
bn_token = 12012290
start_dt = '2024-04-25'
end_dt = '2024-04-26'
time_frame = '3minute'
df = pd.DataFrame(kite.historical_data(bn_token, from_date = start_dt, to_date = end_dt, interval = time_frame, continuous=False, oi=True))

call_token = 12679938
put_token = 12680194
df_call_atm = pd.DataFrame(kite.historical_data(call_token, from_date = start_dt, to_date = end_dt, interval = time_frame, continuous=False, oi=True))
df_put_atm = pd.DataFrame(kite.historical_data(put_token, from_date = start_dt, to_date = end_dt, interval = time_frame, continuous=False, oi=True))

In [120]:
df = date_format(df)
df_call_atm = date_format(df_call_atm)
df_put_atm = date_format(df_put_atm)

In [121]:
df.head()

,date,open,high,low,close,volume,oi,date_only
0,2024-04-25 09:15:00,48005.40,48236.15,47999.90,48157.95,136770,1639500,2024-04-25
1,2024-04-25 09:18:00,48161.00,48163.20,48130.00,48155.30,49470,1649085,2024-04-25
2,2024-04-25 09:21:00,48153.15,48164.80,48139.80,48142.00,22890,1657440,2024-04-25
3,2024-04-25 09:24:00,48142.00,48169.95,48142.00,48163.80,21135,1660470,2024-04-25
4,2024-04-25 09:27:00,48162.25,48195.75,48162.25,48189.20,18930,1659585,2024-04-25


In [122]:
df_merged = df.merge(df_call_atm, on = 'date', suffixes = ("_fut", '_strike'))

In [123]:
import pandas as pd

# Assuming df_merged is your DataFrame with columns 'date', 'close_fut', and 'close_strike'

# Calculate the day-over-day changes for 'close_fut' and 'close_strike'
df_merged['change_close_fut'] = df_merged['close_fut'].diff()
df_merged['change_close_strike'] = df_merged['close_strike'].diff()

# Calculate the sensitivity of 'close_strike' relative to 'close_fut'
# We'll avoid division by zero by replacing 0 with NaN in the denominator
df_merged['sensitivity'] = df_merged['change_close_strike'] / df_merged['change_close_fut'].replace({0: pd.NA})


# Optional: if you want to know the sensitivity for a specific 1 point change in 'close_fut',
# you can calculate the average sensitivity where 'change_close_fut' is non-zero
average_sensitivity = df_merged[df_merged['change_close_fut'] != 0]['sensitivity'].mean()


In [126]:
df_merged[
    ['date','close_fut','close_strike','change_close_fut','change_close_strike','sensitivity']].tail(10)

,date,close_fut,close_strike,change_close_fut,change_close_strike,sensitivity
221,2024-04-26 14:03:00,48452.20,357.50,1.30,1.10,0.846154
222,2024-04-26 14:06:00,48451.90,355.85,-0.30,-1.65,5.5
223,2024-04-26 14:09:00,48495.05,373.95,43.15,18.10,0.419467
224,2024-04-26 14:12:00,48490.25,372.30,-4.80,-1.65,0.34375
225,2024-04-26 14:15:00,48542.80,403.60,52.55,31.30,0.595623
226,2024-04-26 14:18:00,48527.80,393.25,-15.00,-10.35,0.69
227,2024-04-26 14:21:00,48505.65,379.35,-22.15,-13.90,0.62754
228,2024-04-26 14:24:00,48485.10,371.60,-20.55,-7.75,0.377129
229,2024-04-26 14:27:00,48471.10,364.00,-14.00,-7.60,0.542857
230,2024-04-26 14:30:00,48492.00,375.45,20.90,11.45,0.547847


In [90]:
df_merged

,date,open_fut,high_fut,low_fut,close_fut,volume_fut,oi_fut,date_only_fut,open_sp,high_sp,low_sp,close_sp,volume_sp,oi_sp,date_only_sp
0,2024-04-25 09:15:00,48005.40,48236.15,47999.90,48157.95,136770,1639500,2024-04-25,161.90,211.60,142.05,171.95,294375,275475,2024-04-25
1,2024-04-25 09:18:00,48161.00,48163.20,48130.00,48155.30,49470,1649085,2024-04-25,171.95,174.75,157.20,160.95,209085,336420,2024-04-25
2,2024-04-25 09:21:00,48153.15,48164.80,48139.80,48142.00,22890,1657440,2024-04-25,160.85,166.75,154.45,157.55,132000,374400,2024-04-25
3,2024-04-25 09:24:00,48142.00,48169.95,48142.00,48163.80,21135,1660470,2024-04-25,157.55,165.95,155.25,165.05,141435,401685,2024-04-25
4,2024-04-25 09:27:00,48162.25,48195.75,48162.25,48189.20,18930,1659585,2024-04-25,165.05,183.10,164.70,178.95,161685,423285,2024-04-25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201,2024-04-26 13:03:00,48638.85,48640.00,48622.00,48629.80,2880,1821315,2024-04-26,336.70,339.95,330.75,338.60,177960,1671945,2024-04-26
202,2024-04-26 13:06:00,48627.40,48639.75,48611.00,48613.35,3885,1821330,2024-04-26,338.65,340.65,325.85,332.25,228060,1681230,2024-04-26
203,2024-04-26 13:09:00,48615.75,48639.00,48613.35,48630.15,1875,1821840,2024-04-26,332.25,339.95,330.00,338.25,186660,1663995,2024-04-26
204,2024-04-26 13:12:00,48630.15,48660.00,48630.15,48645.90,9000,1822500,2024-04-26,338.25,353.15,337.40,343.00,417405,1670610,2024-04-26


In [ ]:
df = df[df.date_only == "2024-04-26"]
df.set_index('date', inplace=True)

# VWAP Calculation (Manual)
cumulative_vp = (df['volume'] * (df['high'] + df['low'] + df['close']) / 3).cumsum()
cumulative_volume = df['volume'].cumsum()
df['vwap'] = cumulative_vp / cumulative_volume

# Calculate ATR using TA-Lib
df['atr'] = talib.ATR(df['high'].values, df['low'].values, df['close'].values, timeperiod=10)

# Supertrend Calculation
factor = 2
df['basic_upperband'] = (df['high'] + df['low']) / 2 + (factor * df['atr'])
df['basic_lowerband'] = (df['high'] + df['low']) / 2 - (factor * df['atr'])
df['supertrend'] = 0.0
df['in_uptrend'] = True

for current in range(1, len(df.index)):
    previous = current - 1
    if df['close'][current] > df['basic_upperband'][previous]:
        df['in_uptrend'][current] = True
    elif df['close'][current] < df['basic_lowerband'][previous]:
        df['in_uptrend'][current] = False

    if df['in_uptrend'][current]:
        df['supertrend'][current] = df['basic_lowerband'][current]
    else:
        df['supertrend'][current] = df['basic_upperband'][current]

In [66]:
df.round(2).tail(5)

,open,high,low,close,volume,oi,date_only,vwap,atr,basic_upperband,basic_lowerband,supertrend,in_uptrend
date,,,,,,,,,,,,,
2024-04-26 12:48:00,48596.00,48599.00,48583.35,48593.95,5835,1817385,2024-04-26,48619.72,29.79,48650.76,48531.59,48531.59,True
2024-04-26 12:51:00,48593.95,48646.00,48593.10,48630.90,10860,1816890,2024-04-26,48619.76,32.10,48683.75,48555.35,48555.35,True
2024-04-26 12:54:00,48638.00,48650.00,48634.85,48644.00,14130,1818525,2024-04-26,48620.09,30.80,48704.03,48580.82,48580.82,True
2024-04-26 12:57:00,48638.65,48650.00,48615.00,48628.65,8760,1817610,2024-04-26,48620.19,31.22,48694.94,48570.06,48570.06,True
2024-04-26 13:00:00,48628.65,48628.65,48620.05,48620.95,525,1817610,2024-04-26,48620.19,28.96,48682.27,48566.43,48566.43,True


In [ ]:
import pandas as pd
import numpy as np
import talib

# # Example DataFrame
# data = {
#     'date': pd.date_range(start='2023-01-01', periods=100, freq='D'),
#     'open': np.random.random(100) * 100,
#     'high': np.random.random(100) * 100,
#     'low': np.random.random(100) * 100,
#     'close': np.random.random(100) * 100,
#     'volume': np.random.randint(100, 1000, 100)
# }

# df = pd.DataFrame(data)







In [5]:
inst

,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange
0,12014082,46930,NIFTY24MAYFUT,NIFTY,0.0,2024-05-30,0.0,0.05,25,FUT,NFO-FUT,NFO
1,8961026,35004,NIFTY24JUNFUT,NIFTY,0.0,2024-06-27,0.0,0.05,25,FUT,NFO-FUT,NFO
2,8961794,35007,NIFTY24JULFUT,NIFTY,0.0,2024-07-25,0.0,0.05,25,FUT,NFO-FUT,NFO
3,12012290,46923,BANKNIFTY24MAYFUT,BANKNIFTY,0.0,2024-05-29,0.0,0.05,15,FUT,NFO-FUT,NFO
4,8965122,35020,BANKNIFTY24JUNFUT,BANKNIFTY,0.0,2024-06-26,0.0,0.05,15,FUT,NFO-FUT,NFO
...,...,...,...,...,...,...,...,...,...,...,...,...
34230,37560578,146721,ZYDUSLIFE24JUL1060PE,ZYDUSLIFE,0.0,2024-07-25,1060.0,0.05,900,PE,NFO-OPT,NFO
34231,37560834,146722,ZYDUSLIFE24JUL1080CE,ZYDUSLIFE,0.0,2024-07-25,1080.0,0.05,900,CE,NFO-OPT,NFO
34232,37561090,146723,ZYDUSLIFE24JUL1080PE,ZYDUSLIFE,0.0,2024-07-25,1080.0,0.05,900,PE,NFO-OPT,NFO
34233,37561858,146726,ZYDUSLIFE24JUL1120CE,ZYDUSLIFE,0.0,2024-07-25,1120.0,0.05,900,CE,NFO-OPT,NFO


## stock selection 

first get list of shares to check 
1. eod analyser, futures data at day level to check last 3 to 4 days pattern
2. futures data of current day, to look some buildup in the morning
3. future price data to check OL, OH
4. future price to check how many high or low breaks


In [7]:
from tqdm import tqdm
def fetch_market_data(kite, expiry_march, start_dt, end_dt, time_frame="minute"):
    df_expiry_hist_data = pd.DataFrame()
    strike_symbol_dict = dict(
        zip(expiry_march.instrument_token, expiry_march["name"])
    )
    inst_expiry = expiry_march["instrument_token"].unique().tolist()
    for i in tqdm(inst_expiry):
        try:
            df = pd.DataFrame(
                kite.historical_data(
                    i,
                    from_date=start_dt,
                    to_date=end_dt,
                    interval=time_frame,
                    continuous=False,
                    oi=True,
                )
            )
            df["symbol"] = strike_symbol_dict[i]
            df_expiry_hist_data = df_expiry_hist_data.append(df)
        except Exception as e:
            print(e)
            continue
    df_expiry_hist_data["date"] = pd.to_datetime(
        df_expiry_hist_data["date"]
    ).dt.tz_localize(None)
    df_expiry_hist_data["date_only"] = pd.to_datetime(
        df_expiry_hist_data["date"].dt.date
    )
    return df_expiry_hist_data

In [6]:

def classify_action(row):
    if pd.isna(row['oi_diff']) or pd.isna(row['close_diff']):
        return None  # Ignore the first row for each symbol as it has no previous day to compare
    if row['oi_diff'] > 0 and row['close_diff'] > 0:
        return 'Long Buildup'
    elif row['oi_diff'] > 0 and row['close_diff'] < 0:
        return 'Short Buildup'
    elif row['oi_diff'] < 0 and row['close_diff'] > 0:
        return 'Short Covering'
    elif row['oi_diff'] < 0 and row['close_diff'] < 0:
        return 'Long Unwinding'
    else:
        return 'No Change'  # This covers cases where either oi or close doesn't change


def get_buildup_tag(df_data):
    # Sorting by symbol and date to ensure the data is in the right order
    df_data.sort_values(by=['symbol', 'date'], inplace=True)

    # Calculate differences and percentage changes
    df_data['oi_diff'] = df_data.groupby('symbol')['oi'].diff()
    df_data['oi_pct_chg'] = df_data.groupby('symbol')['oi'].pct_change() * 100
    df_data['close_diff'] = df_data.groupby('symbol')['close'].diff()
    df_data['close_pct_chg'] = df_data.groupby('symbol')['close'].pct_change() * 100
    # Apply function
    df_data['Market_Action'] = df_data.apply(classify_action, axis=1)

    df_data = df_data.round(2)
    return df_data


def get_majority_buildup_flag(df_titan):
    # Sample df_titan loading
    df_titan['volume'] = pd.to_numeric(df_titan['volume'])

    # Calculate percentage change in volume
    # df_titan['perc_change_volume'] = df_titan.groupby('symbol')['volume'].pct_change() * 100
    df_titan['perc_change_volume'] = df_titan['volume']
    mean_volume = df_titan['perc_change_volume'].mean()
    std_volume = df_titan['perc_change_volume'].std()
    df_titan['std_flag'] = ((df_titan['perc_change_volume'] > mean_volume + 2 * std_volume) | (df_titan['perc_change_volume'] < mean_volume - 2 * std_volume))

    Q1 = df_titan['perc_change_volume'].quantile(0.25)
    Q3 = df_titan['perc_change_volume'].quantile(0.75)
    IQR = Q3 - Q1
    df_titan['iqr_flag'] = ((df_titan['perc_change_volume'] < Q1 - 1.5 * IQR) | (df_titan['perc_change_volume'] > Q3 + 1.5 * IQR))


    df_titan['z_score'] = (df_titan['perc_change_volume'] - mean_volume) / std_volume
    df_titan['z_flag'] = (df_titan['z_score'].abs() > 2)

    df_titan['majority_flag'] = df_titan[['std_flag', 'iqr_flag', 'z_flag']].sum(axis=1) >= 2
    return df_titan

In [10]:
inst = pd.DataFrame(kite.instruments("NFO"))
april_fut_inst = inst[inst["tradingsymbol"].str.contains('24JUNFUT')]

start_dt = "2024-0-20"
end_dt = "2024-05-07" 
time_frame = "day"
df_day = fetch_market_data(kite,april_fut_inst, start_dt, end_dt, time_frame=time_frame)

100%|██████████| 187/187 [00:10<00:00, 18.44it/s]


In [54]:
df_day.head()

,date,open,high,low,close,volume,oi,symbol,date_only
0,2024-04-22,22325.00,22531.0,22325.00,22496.55,2541350,5333650,NIFTY,2024-04-22
1,2024-04-23,22549.95,22550.0,22465.00,22485.70,3431850,7013650,NIFTY,2024-04-23
2,2024-04-24,22535.00,22596.0,22479.05,22509.75,4705350,8788750,NIFTY,2024-04-24
3,2024-04-25,22435.20,22715.0,22435.15,22653.95,11645250,11448050,NIFTY,2024-04-25
4,2024-04-26,22684.00,22705.0,22530.00,22556.00,6242950,10399450,NIFTY,2024-04-26


In [55]:
start_dt_td = "2024-04-20"
end_dt_td = "2024-05-07" 
time_frame = "30minute"
df_data_min = fetch_market_data(kite,april_fut_inst, start_dt_td, end_dt_td, time_frame=time_frame)

  0%|          | 0/187 [00:00<?, ?it/s]

100%|██████████| 187/187 [00:13<00:00, 14.26it/s]


In [56]:
date_list = ["2024-05-06", "2024-05-07"]
# [df_data_min['symbol'] == "TITAN"].
df_stock_select = pd.DataFrame()
for sy in tqdm(df_data_min.symbol.unique()):
    try:
        df_day_ind = df_day.query("symbol == @sy")
        df_day_ind = df_day_ind.sort_values('date_only', ascending = False)

        df_titan = df_data_min.query("symbol ==@sy")
        df_titan = df_titan[df_titan['date_only'].isin(date_list)]

        # current day fut buildup 
        df_titan = get_buildup_tag(df_titan)
        df_titan = get_majority_buildup_flag(df_titan)

        # OH and OL logic
        df_titan = df_titan.query("date_only == '2024-05-07'")
        df_titan['OL'] = df_titan[df_titan['date'].dt.time.between(pd.to_datetime('09:15').time(), pd.to_datetime('09:15').time())].apply(lambda x: x['open'] == x['low'],axis =1 )
        df_titan['OH'] = df_titan[df_titan['date'].dt.time.between(pd.to_datetime('09:15').time(), pd.to_datetime('09:15').time())].apply(lambda x: x['open'] == x['high'],axis =1 )
        df_titan['OL'] = df_titan['OL'].ffill()
        df_titan['OH'] = df_titan['OH'].ffill()

        # 9 day high and low breakout logic
        df_min_temp = df_titan.copy()
        df_min_temp = df_min_temp[df_min_temp['date'].dt.time.between(pd.to_datetime('09:15').time(), pd.to_datetime('10:00').time())]
        df_min_temp = df_min_temp.resample('D', on='date').agg({
            'open': 'first',
            'high': 'max',
            'low': 'min',
            'close': 'last'
        }).dropna().reset_index()

        today_high = df_min_temp['high'].iloc[0]
        today_low = df_min_temp['low'].iloc[0]
        df_prev = df_day_ind[df_day_ind["date"] != "2024-05-06"]
        # Initialize counters for consecutive breaks
        consecutive_high_breaks = 0
        consecutive_low_breaks = 0

        # Iterate over the rows in reverse (from most recent to least)
        for index, row in df_prev.iloc[:].iterrows():
            if row['high'] < today_high:
                consecutive_high_breaks += 1
            else:
                break  # Stop counting if a day does not break today's high

        for index, row in df_prev.iloc[:].iterrows():

            if row['low'] > today_low:
                consecutive_low_breaks += 1
            else:
                break  # Stop counting if a day does not break today's low
        df_titan['day_count_high'] = consecutive_high_breaks
        df_titan['day_count_low'] = consecutive_low_breaks
        df_stock_select = df_stock_select.append(df_titan)
        # break
    except:
        print(sy)
        continue


  0%|          | 0/187 [00:00<?, ?it/s]

100%|██████████| 187/187 [00:10<00:00, 18.30it/s]


In [57]:
df_stock_select.to_clipboard(index = False)

In [49]:
df_stock_select.query("OL == True").symbol.unique()

array(['ABB', 'APOLLOHOSP', 'AXISBANK', 'BALRAMCHIN', 'BATAINDIA',
       'COALINDIA', 'CROMPTON', 'DEEPAKNTR', 'DLF', 'EICHERMOT',
       'GODREJPROP', 'HDFCBANK', 'INDHOTEL', 'JUBLFOOD', 'METROPOLIS',
       'MFSL', 'MIDCPNIFTY', 'NMDC', 'POWERGRID', 'RELIANCE', 'SBILIFE',
       'SIEMENS', 'SRF', 'SYNGENE'], dtype=object)

In [50]:
df_stock_select.query("OH == True").symbol.unique()

array(['ABCAPITAL', 'ABFRL', 'ADANIENT', 'ALKEM', 'APOLLOTYRE',
       'ASIANPAINT', 'ASTRAL', 'BALKRISIND', 'BHARATFORG', 'BIOCON',
       'CANFINHOME', 'CHOLAFIN', 'CUB', 'CUMMINSIND', 'DABUR',
       'FEDERALBNK', 'GMRINFRA', 'GRASIM', 'HDFCLIFE', 'ICICIBANK', 'IGL',
       'INDIGO', 'JINDALSTEL', 'LTF', 'LTTS', 'LUPIN', 'MRF', 'PAGEIND',
       'PETRONET', 'PFC', 'POLYCAB', 'PVRINOX', 'RBLBANK', 'SHRIRAMFIN',
       'VEDL'], dtype=object)

In [52]:
df_stock_select.to_excel("stock_select.xlsx",index = False)

In [116]:
df_stock_select.query("OH == True").query("majority_flag == True").query("day_count_high == 1").query("day_count_low == 1").query("Market_Action == 'Long Buildup'")

,date,open,high,low,close,volume,oi,symbol,date_only,oi_diff,...,perc_change_volume,std_flag,iqr_flag,z_score,z_flag,majority_flag,OL,OH,day_count_high,day_count_low
625,2024-04-12 11:20:00,4840.75,4848.00,4840.00,4841.20,20200,1086600,ALKEM,2024-04-12,6600.0,...,20200,True,True,3.291733,True,True,False,True,0,1
628,2024-04-12 11:35:00,4837.00,4847.00,4837.00,4838.30,23400,1118400,ALKEM,2024-04-12,14800.0,...,23400,True,True,3.927407,True,True,False,True,0,1
669,2024-04-12 15:00:00,4767.75,4767.75,4743.45,4750.00,17200,1135000,ALKEM,2024-04-12,200.0,...,17200,True,True,2.695788,True,True,False,True,0,1
600,2024-04-12 09:15:00,6490.50,6490.50,6437.30,6471.20,29875,1705125,APOLLOHOSP,2024-04-12,-6250.0,...,29875,True,True,4.826337,True,True,False,True,0,1
601,2024-04-12 09:20:00,6465.95,6465.95,6446.20,6465.65,15500,1711375,APOLLOHOSP,2024-04-12,6250.0,...,15500,True,True,2.002626,True,True,False,True,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
674,2024-04-12 15:25:00,3628.90,3632.00,3626.45,3627.10,82425,5798100,TITAN,2024-04-12,1925.0,...,82425,True,True,3.543100,True,True,False,True,0,1
600,2024-04-12 09:15:00,1339.05,1339.05,1330.60,1334.45,155400,6912000,VOLTAS,2024-04-12,-524400.0,...,155400,True,True,2.570002,True,True,False,True,0,0
669,2024-04-12 15:00:00,1320.95,1321.15,1311.20,1315.20,168000,7314600,VOLTAS,2024-04-12,-1200.0,...,168000,True,True,2.867631,True,True,False,True,0,0
670,2024-04-12 15:05:00,1315.20,1318.05,1312.10,1313.70,153000,7349400,VOLTAS,2024-04-12,34800.0,...,153000,True,True,2.513311,True,True,False,True,0,0


In [ ]:
# significant_changes = df_titan[df_titan['majority_flag']]
# significant_changes = significant_changes[significant_changes['date'].dt.date == pd.to_datetime('2024-04-12').date()]
# significant_changes = significant_changes[significant_changes['date'].dt.time.between(pd.to_datetime('09:15').time(), pd.to_datetime('10:00').time())]

## stock 9:25

1. 9:20 top gainer/ loser (Futures)
2. more then 2% move 
3. nse spurtes OI -> more then 7%
4. nifty 10 min candle  -> check 

5. entry:
	1. 9:25 mark high, if 9:25 high break, enter the trade
	 cond:
		dont trade if 50% pull back 
		enter before 10:30 
		monthly expire -> 2 din phale and 2 din bad 
		
	exit:
	1. 2 candle below 8 sma 
	
other condition
1. 9 day high or low 
2. OL or OH
3. day future some good buildup 
4. prev 3 or 4 day fut
5. pcr cutoff for above and below 
i have observed about PCR if its above 1 then most probably its uptrend 
and if its below .5 then down trend 



#can we check the pcr value and do the fundatmental analysis 


In [5]:
# futures some high OI
# last few days oi buildup 
# 9 day high
# OL or OH

In [10]:
from tqdm import tqdm

def fetch_market_data(kite, expiry_march, start_dt, end_dt, time_frame="minute"):
    df_expiry_hist_data = pd.DataFrame()
    strike_symbol_dict = dict(
        zip(expiry_march.instrument_token, expiry_march["name"])
    )
    inst_expiry = expiry_march["instrument_token"].unique().tolist()
    for i in tqdm(inst_expiry):
        try:
            df = pd.DataFrame(
                kite.historical_data(
                    i,
                    from_date=start_dt,
                    to_date=end_dt,
                    interval=time_frame,
                    continuous=False,
                    oi=True,
                )
            )
            df["symbol"] = strike_symbol_dict[i]
            df_expiry_hist_data = df_expiry_hist_data.append(df)
        except Exception as e:
            print(e)
            continue
    df_expiry_hist_data["date"] = pd.to_datetime(
        df_expiry_hist_data["date"]
    ).dt.tz_localize(None)
    df_expiry_hist_data["date_only"] = pd.to_datetime(
        df_expiry_hist_data["date"].dt.date
    )
    return df_expiry_hist_data

In [11]:
# select the stocks list which are in futures only
inst = pd.DataFrame(kite.instruments("NFO"))
inst_nse = pd.DataFrame(kite.instruments("NSE"))

april_fut_inst = inst[inst["tradingsymbol"].str.contains('24JULFUT')]


In [12]:

start_dt = "2024-07-01"
end_dt = "2024-07-25" 
time_frame = "day"

start_dt_td = "2024-07-01"
end_dt_td = "2024-07-25" 
time_frame_td = "5minute"

df_day_fut = fetch_market_data(kite,april_fut_inst, start_dt, end_dt,time_frame)
df_data_min_fut = fetch_market_data(kite,april_fut_inst, start_dt_td, end_dt_td,time_frame_td)

inst_nse = pd.DataFrame(kite.instruments("NSE"))
stock_token = inst_nse[inst_nse.tradingsymbol.isin(df_day_fut.symbol.unique())]

df_day = fetch_market_data(kite,stock_token, start_dt, end_dt, time_frame)
df_data_min = fetch_market_data(kite,stock_token, start_dt_td, end_dt_td,time_frame_td)

100%|██████████| 181/181 [00:54<00:00,  3.32it/s]


In [13]:
# getting fut oi % chg at 9:20 
list_of_dates_fut = list(df_data_min_fut["date_only"].unique())

df_data_min_fut["prev_date"] = df_data_min_fut["date_only"].apply(
    lambda x: (
        np.nan
        if list_of_dates_fut.index(x) == 0
        else list_of_dates_fut[list_of_dates_fut.index(x) - 1]
    )
)

df_data_min_fut = df_data_min_fut.merge(
    df_day_fut[["date_only", "symbol", "oi"]],
    how="left",
    left_on=["symbol", "prev_date"],
    right_on=["symbol", "date_only"],
    suffixes=("_min", "_day"),
)


df_data_min_fut["per_chg"] = (
    (df_data_min_fut["oi_min"] - df_data_min_fut["oi_day"]) / df_data_min_fut["oi_day"]
) * 100

df_data_min_fut.set_index("date", inplace=True)
df_oi_fut = df_data_min_fut.between_time("9:20",'9:20')#.dropna()


In [14]:
# getting price % chg at 9:20 
list_of_dates = list(df_data_min["date_only"].unique())

df_data_min["prev_date"] = df_data_min["date_only"].apply(
    lambda x: (
        np.nan
        if list_of_dates.index(x) == 0
        else list_of_dates[list_of_dates.index(x) - 1]
    )
)

df_data_min = df_data_min.merge(
    df_day[["date_only", "symbol", "close"]],
    how="left",
    left_on=["symbol", "prev_date"],
    right_on=["symbol", "date_only"],
    suffixes=("_min", "_day"),
)

df_data_min["per_chg"] = (
    (df_data_min["close_min"] - df_data_min["close_day"]) / df_data_min["close_day"]
) * 100

df_data_min.set_index("date", inplace=True)
df_nine_filt = df_data_min.between_time("9:20",'9:20')#.dropna()
df_nine_filt.reset_index(inplace=True)
df_nine_filt = df_nine_filt.merge(inst_nse[['tradingsymbol','name']], how ='left', left_on = 'symbol',right_on = 'name')
df_nine_filt = df_nine_filt.drop(['name','symbol'],axis =1 ).rename(columns= {'tradingsymbol':'symbol'})

In [15]:
(
    df_nine_filt
    .merge(df_oi_fut.reset_index()[['date','symbol','per_chg']],
            how = 'left',
            on = ['date','symbol'],
            suffixes = ("_price","_oi")
              
              )

).to_clipboard()

In [19]:
df_nine_filt[(df_nine_filt.per_chg > 2 )|( df_nine_filt.per_chg < -2)].to_clipboard()